In [0]:
import torch
import torch.nn as nn
import torch.utils as utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [0]:
# Setting Hyperparameters

epoch = 10
batch_size =100
learning_rate = 0.0005

# Downloading CIFAR 10 Data

cifar10_train = dset.CIFAR10("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
cifar10_test  = dset.CIFAR10("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

# Set Data Loader(input pipeline)

train_loader = torch.utils.data.DataLoader(dataset=cifar10_train,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=cifar10_test,batch_size=batch_size,shuffle=True)

In [0]:
# Defining encoder class
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder,self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(3,32,3,padding=1),   # batch x 32 x 32 x 32
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32,32,3,padding=1),   # batch x 32 x 32 x 32
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.Conv2d(32,64,3,padding=1),  # batch x 64 x 32 x 32
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.Conv2d(64,64,3,padding=1),  # batch x 64 x 32 x 32
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.MaxPool2d(2,2)   # batch x 64 x 16 x 16
        )
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,128,3,padding=1),  # batch x 128 x 16 x 16
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.Conv2d(128,128,3,padding=1),  # batch x 128 x 16 x 16
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.MaxPool2d(2,2),
                        nn.Conv2d(128,256,3,padding=1),  # batch x 256 x 8 x 8
                        nn.ReLU()
        )
        
                
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(batch_size, -1)
        return out
    
encoder = Encoder().cuda()

In [0]:
#Adding noise to the input image
def add_noise(img):
    noise = torch.randn(img.size()) * 0.2
    noisy_img = img + noise
    return noisy_img

In [0]:
# Defining Decoder class
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder,self).__init__()
        self.layer1 = nn.Sequential(
                        nn.ConvTranspose2d(256,128,3,2,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.ConvTranspose2d(128,128,3,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(128),
                        nn.ConvTranspose2d(128,64,3,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(64),
                        nn.ConvTranspose2d(64,64,3,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(64)
        )
        self.layer2 = nn.Sequential(
                        nn.ConvTranspose2d(64,32,3,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.ConvTranspose2d(32,32,3,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(32),
                        nn.ConvTranspose2d(32,3,3,2,1,1),
                        nn.ReLU()
        )
        
    def forward(self,x):
        out = x.view(batch_size,256,8,8)
        out = self.layer1(out)
        out = self.layer2(out)
        return out

decoder = Decoder().cuda()

In [0]:
# Check output of autoencoder

for image,label in train_loader:
    image = image.cuda()
    
    output = encoder(image)
    output = decoder(output)
    print(output.size())
    break

In [0]:
# loss func and optimizer
# we compute reconstruction after decoder so use Mean Squared Error
# In order to use multi parameters with one optimizer,
# concat parameters after changing into list

parameters = list(encoder.parameters())+ list(decoder.parameters())
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(parameters, lr=learning_rate)

In [0]:
# training encoder and decoder on cifar 10 data
# save and load model
if not os.path.exists('./model'):
    os.mkdir('./model')
try:
    encoder, decoder = torch.load('./model/deno_autoencoder.pkl')
    print("\n--------model restored--------\n")
except:
    print("\n--------model not restored--------\n")
    pass
epoch=1
for i in range(epoch):
    for image,label in train_loader:
        image_n = add_noise(image)
        image = image.cuda()
        image_n = image_n.cuda()
        #label = label.float().cuda()
        optimizer.zero_grad()
        output = encoder(image_n)
        output = decoder(output)
        loss = loss_func(output,image)
        loss.backward()
        optimizer.step()
    print('epoch [{}/{}], loss:{:.4f}'
          .format(i + 1, epoch, loss.item()))
        
                
torch.save([encoder,decoder],'./model/deno_autoencoder.pkl')
print(loss)

In [0]:
#Original image from the training set
img = image[1].cpu()
#Noisy image after adding noise to original image
noisy_img = image_n[1].cpu()
#Output image after encoder and decoder
output_img = output[1].cpu()

origin = img.numpy()
noisy_inp = noisy_img.numpy()
out = output_img.detach().numpy()

print('Original image from dataset is:')
plt.imshow(origin[0])
plt.show()

print('Image after adding noise is:')
plt.imshow(noisy_inp[0])
plt.show()

print('Reconstructed image is:')
plt.imshow(out[0])
plt.show()

In [0]:
#latent space classification
#The latent space data is given an input to the SVM classfier
from sklearn import svm
correct=0
knn=svm.SVC(kernel='linear')
with torch.no_grad():
  for image,label in train_loader:
    image = image.cuda()
    output = encoder(image)
    classifier=knn.fit(output.cpu(),label)

for data, target in test_loader:
      image = data.cuda()
      output = encoder(image)
      output1 = Variable(output, requires_grad = False)
      predicted= knn.predict(output1.cpu())
      for i in range(len(predicted)):
        if(predicted[i]==target[i]):
           correct+= 1
      
accuracy = 100. * correct / float(len(test_loader.dataset))
print('accuracy on testing data is '+str(accuracy)+ '%')